# Sea Ice Concentration by season Data Processing from CESM2 Hindcast
Process data for Southern Ocean SIC in different seasons for validation
- Notebook by Alice DuVivier (NCAR)
- Note: this notebook needs to load the utils.py file
- August 2025

In [1]:
import utils
import xarray as xr
import numpy as np
import pandas as pd
import dask
from distributed import Client
from ncar_jobqueue import NCARCluster
from glob import glob
import intake
import importlib
import pop_tools
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs

/glade/work/duvivier/miniconda3/envs/analysis3/lib/python3.7/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


## Spin up DASK cluster

In [2]:
# Create our NCAR Cluster - which uses PBSCluster under the hood
cluster = NCARCluster(walltime='2:00:00', cores=1, processes=1, memory='100 GB', 
                      resource_spec='select=1:ncpus=1:mem=100GB',interface='ext')

# Spin up 32 workers
cluster.scale(32)

# Assign the cluster to our Client
client = Client(cluster)

In [3]:
client
# cluster.close()

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/duvivier/PBS/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/duvivier/PBS/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://128.117.208.173:44173,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/duvivier/PBS/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


## Read in grid data

In [4]:
# directory path
dir_in = '/glade/u/home/duvivier/masks/'

In [5]:
# load ocean grid
fin = 'ocn_grid_gx1v7.nc'
ds_grid = xr.open_mfdataset(dir_in+fin,decode_times=False)
ds_grid['nlat'] = ds_grid.nlat
ds_grid['nlon'] = ds_grid.nlon
ds_grid

<xarray.Dataset>
Dimensions:                 (moc_comp: 3, transport_comp: 5, transport_reg: 2, z_t: 60, z_t_150m: 15, z_w: 60, z_w_top: 60, z_w_bot: 60, lat_aux_grid: 395, moc_z: 61, nlat: 384, nlon: 320, time: 1188, d2: 2)
Coordinates: (12/14)
  * z_t                     (z_t) float32 500.0 1.5e+03 ... 5.125e+05 5.375e+05
  * z_t_150m                (z_t_150m) float32 500.0 1.5e+03 ... 1.45e+04
  * z_w                     (z_w) float32 0.0 1e+03 2e+03 ... 5e+05 5.25e+05
  * z_w_top                 (z_w_top) float32 0.0 1e+03 2e+03 ... 5e+05 5.25e+05
  * z_w_bot                 (z_w_bot) float32 1e+03 2e+03 ... 5.25e+05 5.5e+05
  * lat_aux_grid            (lat_aux_grid) float32 -79.49 -78.95 ... 89.47 90.0
    ...                      ...
    ULAT                    (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    TLONG                   (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    TLAT                    (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
  * time                    (time) float32 31.0 59.0 90.0 ... 3.61e+04 3.614e+04
  * nlat                    (nlat) int64 0 1 2 3 4 5 ... 378 379 380 381 382 383
  * nlon                    (nlon) int64 0 1 2 3 4 5 ... 314 315 316 317 318 319
Dimensions without coordinates: moc_comp, transport_comp, transport_reg, d2
Data variables: (12/58)
    moc_components          (moc_comp) |S384 dask.array<chunksize=(3,), meta=np.ndarray>
    transport_components    (transport_comp) |S384 dask.array<chunksize=(5,), meta=np.ndarray>
    transport_regions       (transport_reg) |S384 dask.array<chunksize=(2,), meta=np.ndarray>
    dz                      (z_t) float32 dask.array<chunksize=(60,), meta=np.ndarray>
    dzw                     (z_w) float32 dask.array<chunksize=(60,), meta=np.ndarray>
    KMT                     (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    ...                      ...
    nsurface_u              float64 ...
    time_bound              (time, d2) float64 dask.array<chunksize=(1188, 2), meta=np.ndarray>
    QFLUX                   (time, nlat, nlon) float32 dask.array<chunksize=(1188, 384, 320), meta=np.ndarray>
    SHF                     (time, nlat, nlon) float32 dask.array<chunksize=(1188, 384, 320), meta=np.ndarray>
    SHF_QSW                 (time, nlat, nlon) float32 dask.array<chunksize=(1188, 384, 320), meta=np.ndarray>
    TEMP                    (time, z_t, nlat, nlon) float32 dask.array<chunksize=(1188, 60, 384, 320), meta=np.ndarray>
Attributes:
    title:             b.e21.B1850.f09_g17.CMIP6-piControl.001
    history:           none
    Conventions:       CF-1.0; http://www.cgd.ucar.edu/cms/eaton/netcdf/CF-cu...
    time_period_freq:  month_1
    model_doi_url:     https://doi.org/10.5065/D67H1H0V
    contents:          Diagnostic and Prognostic Variables
    source:            CCSM POP2, the CCSM Ocean Component
    revision:          $Id: tavg.F90 89644 2018-08-04 14:26:01Z klindsay $
    calendar:          All years have exactly  365 days.
    start_time:        This dataset was created on 2018-08-09 at 18:18:26.3
    cell_methods:      cell_methods = time: mean ==> the variable values are ...

In [6]:
# load sea ice masks
fin = 'REGION_MASK_gx1v7.nc'
ds_mask = xr.open_mfdataset(dir_in+fin,decode_times=False)
ds_mask = ds_mask.isel(time=0)
ds_mask = ds_mask.rename({'lat':'nlat','lon': 'nlon'})
ds_mask['nlat'] = ds_grid.nlat
ds_mask['nlon'] = ds_grid.nlon
ds_mask

<xarray.Dataset>
Dimensions:       (nlon: 320, nlat: 384)
Coordinates:
  * nlon          (nlon) int64 0 1 2 3 4 5 6 7 ... 313 314 315 316 317 318 319
  * nlat          (nlat) int64 0 1 2 3 4 5 6 7 ... 377 378 379 380 381 382 383
Data variables: (12/18)
    Lab_mask      (nlat, nlon) float32 dask.array<chunksize=(384, 320), meta=np.ndarray>
    GIN_mask      (nlat, nlon) float32 dask.array<chunksize=(384, 320), meta=np.ndarray>
    Bar_mask      (nlat, nlon) float32 dask.array<chunksize=(384, 320), meta=np.ndarray>
    ArcOc_mask    (nlat, nlon) float32 dask.array<chunksize=(384, 320), meta=np.ndarray>
    Sib_mask      (nlat, nlon) float32 dask.array<chunksize=(384, 320), meta=np.ndarray>
    Beau_mask     (nlat, nlon) float32 dask.array<chunksize=(384, 320), meta=np.ndarray>
    ...            ...
    Ross_mask     (nlat, nlon) float32 dask.array<chunksize=(384, 320), meta=np.ndarray>
    Ind_mask      (nlat, nlon) float32 dask.array<chunksize=(384, 320), meta=np.ndarray>
    Pac_mask      (nlat, nlon) float32 dask.array<chunksize=(384, 320), meta=np.ndarray>
    BAm_mask      (nlat, nlon) float32 dask.array<chunksize=(384, 320), meta=np.ndarray>
    nh_mask       (nlat, nlon) float32 dask.array<chunksize=(384, 320), meta=np.ndarray>
    sh_mask       (nlat, nlon) float32 dask.array<chunksize=(384, 320), meta=np.ndarray>

In [10]:
# crop data to just the southern hemisphere below 60S
ds_grid = ds_grid.isel(nlat=slice(0,55))
ds_mask = ds_mask.isel(nlat=slice(0,55))

## Read in Hindcast data

In [12]:
## Enter variable of interest
var_in = 'aice'

## Enter the latitudes you want to keep:
lat_min = -80
lat_max = -50

In [13]:
dir_in = '/glade/campaign/cesm/development/bgcwg/projects/CESM2-OMIP2-like-4p2z-run/g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.4p2z.002branch/ice/proc/tseries/month_1/'
file_in = 'g.e22.GOMIPECOIAF_JRA-1p4-2018.TL319_g17.4p2z.002branch.cice.h.'+var_in+'.195801-202112.nc'
ds = xr.open_mfdataset(dir_in+file_in,decode_times=False)

In [14]:
# keep only some variables
keep_vars=['z_t','time_bound','z_t_150m','KMT','TLAT','TLONG','time',var_in]
ds = ds.drop([v for v in ds.variables if v not in keep_vars])

In [15]:
# crop to southern ocean only
#ds = ds.isel(nlat=slice(0,36))
ds = ds.where(((ds['TLAT'] <= lat_max) & (ds['TLAT'] >= lat_min)), drop=True)

In [17]:
# check time array
ds.time

<xarray.DataArray 'time' (time: 768)>
array([714336., 714364., 714395., ..., 737604., 737634., 737665.])
Coordinates:
  * time     (time) float64 7.143e+05 7.144e+05 ... 7.376e+05 7.377e+05
Attributes:
    long_name:     model time
    units:         days since 0001-01-01 00:00:00
    calendar:      noleap
    bounds:        time_bounds
    cell_methods:  time: mean

In [18]:
# to make useable time coordinate, create arrays for yyyy-mm-dd (NO LEAP!)
# We know it's monthly data for 1958.01-2021.12, not including leap days.

# create full range of dates at daily frequency
dates = pd.date_range(start='1/1/1958', end='12/31/2021', freq='M')
# check that the # times matches above
print(dates.shape)

(768,)


In [19]:
# assign the new time array
ds['time'] = dates

In [20]:
data = ds.aice

# Compute Seasonal SIC

- JFM, AMJ, JAS, OND

In [21]:
yy_st = 1958
yy_ed = 2021

In [22]:
# get some sizes
years = np.arange(int(yy_st),int(yy_ed),1)

tlen = len(years) 
xlen = len(ds.ni)
ylen = len(ds.nj)
print(tlen,xlen,ylen)

63 320 55


In [23]:
# loop through all years

# make numpy array to fill and specify dimensions
array_avg_jfm = np.zeros([tlen,ylen,xlen])
array_avg_amj = np.zeros([tlen,ylen,xlen])
array_avg_jas = np.zeros([tlen,ylen,xlen])
array_avg_ond = np.zeros([tlen,ylen,xlen])

for yy in years:
    #print('Calculating spring mean for '+str(yy.values))
    # need to specify which spot in the array we'll fill for the decade
    if yy == 1958:
        count = 0
    else: 
        count = count+1  
    # get data subset
    data_sub = data.where(data.time.dt.year == yy, drop = True)
    # JFM - first season - by averaging across all months in this year
    array_avg_jfm[count,:,:] = data_sub.isel(time=slice(0,3)).mean(dim='time')
    # AMJ - second season - by averaging across all months in this year
    array_avg_amj[count,:,:] = data_sub.isel(time=slice(3,6)).mean(dim='time')
    # JAS - first season - by averaging across all months in this year
    array_avg_jas[count,:,:] = data_sub.isel(time=slice(6,9)).mean(dim='time')
    # OND - second season - by averaging across all months in this year
    array_avg_ond[count,:,:] = data_sub.isel(time=slice(9,12)).mean(dim='time')

# convert to xarray
data_jfm = xr.DataArray(array_avg_jfm,dims=('time','nlat','nlon'))
data_jfm['time'] = years
data_jfm['nlat'] = ds_grid.nlat
data_jfm['nlon'] = ds_grid.nlon

data_amj = xr.DataArray(array_avg_amj,dims=('time','nlat','nlon'))
data_amj['time'] = years
data_amj['nlat'] = ds_grid.nlat
data_amj['nlon'] = ds_grid.nlon

data_jas = xr.DataArray(array_avg_jas,dims=('time','nlat','nlon'))
data_jas['time'] = years
data_jas['nlat'] = ds_grid.nlat
data_jas['nlon'] = ds_grid.nlon

data_ond = xr.DataArray(array_avg_ond,dims=('time','nlat','nlon'))
data_ond['time'] = years
data_ond['nlat'] = ds_grid.nlat
data_ond['nlon'] = ds_grid.nlon

## Put together into one dataset

- Netcdf with variable dimension: (year x nlat x nlon) Include TLAT and TLONG as coordinates

In [24]:
ds_out = xr.Dataset()

ds_out['SIC_JFM'] = data_jfm
ds_out['SIC_AMJ'] = data_amj
ds_out['SIC_JAS'] = data_jas
ds_out['SIC_OND'] = data_ond

# change the attributes
ds_out.attrs['author'] = 'Alice DuVivier'
ds_out.attrs['date_processed'] = datetime.now().strftime('%Y-%m-%d')
ds_out.attrs['contents'] = f'Seasonal sea ice fraction in the Southern Ocean from 1958 to 2020 from CESM2 ice/ocean hindcast forced with JRA55.'

ds_out

<xarray.Dataset>
Dimensions:  (time: 63, nlat: 55, nlon: 320)
Coordinates:
  * time     (time) int64 1958 1959 1960 1961 1962 ... 2016 2017 2018 2019 2020
  * nlat     (nlat) int64 0 1 2 3 4 5 6 7 8 9 ... 45 46 47 48 49 50 51 52 53 54
  * nlon     (nlon) int64 0 1 2 3 4 5 6 7 8 ... 312 313 314 315 316 317 318 319
Data variables:
    SIC_JFM  (time, nlat, nlon) float64 nan nan nan nan nan ... 0.0 0.0 0.0 0.0
    SIC_AMJ  (time, nlat, nlon) float64 nan nan nan nan nan ... 0.0 0.0 0.0 0.0
    SIC_JAS  (time, nlat, nlon) float64 nan nan nan nan nan ... 0.0 0.0 0.0 0.0
    SIC_OND  (time, nlat, nlon) float64 nan nan nan nan nan ... 0.0 0.0 0.0 0.0
Attributes:
    author:          Alice DuVivier
    date_processed:  2025-08-06
    contents:        Seasonal sea ice fraction in the Southern Ocean from 195...

In [25]:
# Print the dimensions
print("Dimensions:")
for dim in ds_out.dims:
    print(f"\t{dim}: {ds_out[dim].values.shape}")

# Print the coordinates
print("Coordinates:")
for coord in ds_out.coords:
    print(f"\t{coord}:")
    print(f"\t\t{ds_out.coords[coord].values}")
    
# Print the attributes
print("Attributes:")
for attr in ds_out.attrs:
    print(f"\t{attr}: {ds_out.attrs[attr]}")
    

Dimensions:
	time: (63,)
	nlat: (55,)
	nlon: (320,)
Coordinates:
	time:
		[1958 1959 1960 1961 1962 1963 1964 1965 1966 1967 1968 1969 1970 1971
 1972 1973 1974 1975 1976 1977 1978 1979 1980 1981 1982 1983 1984 1985
 1986 1987 1988 1989 1990 1991 1992 1993 1994 1995 1996 1997 1998 1999
 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013
 2014 2015 2016 2017 2018 2019 2020]
	nlat:
		[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54]
	nlon:
		[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 

## Export and Save

In [26]:
# calculate the size of the dataset in GB
size_gb = ds_out.nbytes / (1024**3)
print(f"The dataset is approximately {size_gb:.2f} GB.")


The dataset is approximately 0.03 GB.


In [27]:
path_out = '/glade/campaign/cgd/ppc/duvivier/cesm2_antarctic_polynya/mpa_analysis/DATA/ecoindex_data/environment/'
file_out = 'CESM2-JRA55-hindcast-SIC_season.nc'
fout = path_out + file_out

In [28]:
# Export the dataset to NetCDF with all attributes and coordinates
ds_out.to_netcdf(fout)

In [29]:
cluster.close()

tornado.application - ERROR - Exception in callback functools.partial(<function TCPServer._handle_connection.<locals>.<lambda> at 0x14743fb413b0>, <Task finished coro=<BaseTCPListener._handle_stream() done, defined at /glade/work/duvivier/miniconda3/envs/analysis3/lib/python3.7/site-packages/distributed/comm/tcp.py:502> exception=ValueError('invalid operation on non-started TCPListener')>)
Traceback (most recent call last):
  File "/glade/work/duvivier/miniconda3/envs/analysis3/lib/python3.7/site-packages/tornado/ioloop.py", line 741, in _run_callback
    ret = callback()
  File "/glade/work/duvivier/miniconda3/envs/analysis3/lib/python3.7/site-packages/tornado/tcpserver.py", line 331, in <lambda>
    gen.convert_yielded(future), lambda f: f.result()
  File "/glade/work/duvivier/miniconda3/envs/analysis3/lib/python3.7/site-packages/distributed/comm/tcp.py", line 508, in _handle_stream
    logger.debug("Incoming connection from %r to %r", address, self.contact_address)
  File "/glade/wo